In [15]:
import sympy

var('q0 q1 L0 L1 l0 l1')

T_0_org = sympy.Matrix([[cos(q0), 0, -sin(q0), L0*cos(q0)], 
                 [0, 1, 0, 0],
                 [sin(q0), 0, cos(q0), L0*sin(q0)],
                 [0, 0, 0, 1]])
T_1_0 = sympy.Matrix([[cos(q1), 0, -sin(q1), L1*cos(q1)], 
                [0, 1, 0, 0],
                [sin(q1), 0, cos(q1), L1*sin(q1)],
                [0, 0, 0, 1]])
com0 = sympy.Matrix([[l0*cos(q0)],
               [0],
               [l0*sin(q0)],
               [1]])
com1 = sympy.Matrix([[l1*cos(q1)],
               [0],
               [l1*sin(q1)],
               [1]])
'''
xee = sympy.Matrix([[0],
              [0],
              [0],
              [1]])
'''
xee = sympy.Matrix([[L1*cos(q1)],
               [0],
               [L1*sin(q1)],
               [1]])

T_1_org = T_0_org.multiply(T_1_0)
T_1_org

Matrix([
[-sin(q0)*sin(q1) + cos(q0)*cos(q1), 0, -sin(q0)*cos(q1) - sin(q1)*cos(q0), L0*cos(q0) - L1*sin(q0)*sin(q1) + L1*cos(q0)*cos(q1)],
[                                 0, 1,                                  0,                                                    0],
[ sin(q0)*cos(q1) + sin(q1)*cos(q0), 0, -sin(q0)*sin(q1) + cos(q0)*cos(q1), L0*sin(q0) + L1*sin(q0)*cos(q1) + L1*sin(q1)*cos(q0)],
[                                 0, 0,                                  0,                                                    1]])

In [16]:
# jacobian of the end effector (linear part)
JEE_v = (T_1_org.multiply(xee)).jacobian([q0, q1])
JEE_v.row_del(3)
JEE_v

Matrix([
[-L0*sin(q0) + L1*(sin(q0)*sin(q1) - cos(q0)*cos(q1))*sin(q1) + L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*cos(q1) - L1*sin(q0)*cos(q1) - L1*sin(q1)*cos(q0), -L1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1))*sin(q1) + L1*(sin(q0)*sin(q1) - cos(q0)*cos(q1))*sin(q1) + 2*L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*cos(q1) - L1*sin(q0)*cos(q1) - L1*sin(q1)*cos(q0)],
[                                                                                                                                                       0,                                                                                                                                                                                               0],
[L0*cos(q0) + L1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1))*cos(q1) + L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*sin(q1) - L1*sin(q0)*sin(q1) + L1*cos(q0)*cos(q1),  2*L1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1))*cos(q1) + L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*sin(q1) - L1*(sin(q0)*cos(q1) + sin(q1)*cos(q

In [19]:
# jacobian of the end effector (angular part)
# y-axis rotation only
JEE_w = sympy.Matrix([[0, 0], 
                [1, 1],
                [0, 0]])

# full jacobian
JEE = JEE_v.row_insert(4, JEE_w)
JEE

Matrix([
[-L0*sin(q0) + L1*(sin(q0)*sin(q1) - cos(q0)*cos(q1))*sin(q1) + L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*cos(q1) - L1*sin(q0)*cos(q1) - L1*sin(q1)*cos(q0), -L1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1))*sin(q1) + L1*(sin(q0)*sin(q1) - cos(q0)*cos(q1))*sin(q1) + 2*L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*cos(q1) - L1*sin(q0)*cos(q1) - L1*sin(q1)*cos(q0)],
[                                                                                                                                                       0,                                                                                                                                                                                               0],
[L0*cos(q0) + L1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1))*cos(q1) + L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*sin(q1) - L1*sin(q0)*sin(q1) + L1*cos(q0)*cos(q1),  2*L1*(-sin(q0)*sin(q1) + cos(q0)*cos(q1))*cos(q1) + L1*(-sin(q0)*cos(q1) - sin(q1)*cos(q0))*sin(q1) - L1*(sin(q0)*cos(q1) + sin(q1)*cos(q

In [34]:
import numpy as np
length0 = 0.3  # leg link 0 length, m
length1 = 0.3  # leg link 1 length, m
l = length1*np.sin(45)+length2*np.sin(45)  # nominal body height
d = 0.5  # jump displacement, m
m = 10  # mass, kg

g = -9.807  # earth gravity
l_s = l*0.75  # assume lift-off stroke to be specified percent of leg length

v0 = np.sqrt(-2*g*d)  # lift-off speed required to jump d meters
a = (v0**2)/(2*l_s)
t = v0/a
F = m*a  # required vertical force, N
print("Vertical force required is", F, "N")
print("Lift-off time based on given stroke is", t, "s")

Vertical force required is 128.0599545363588 N
Lift-off time based on given stroke is 0.2445427233086884 s


In [36]:
F_l = sympy.Matrix([[0],[0],[F],[0],[0],[0]])
F_q = (JEE.T).multiply(F_l)  # torques required given a linear force

q0_nom = -135*np.pi/180  # just guesses
q1_nom = 90*np.pi/180

Torques = F_q.subs({q0:q0_nom, q1:q1_nom, L0:length0, L1:length1})  # torques required for each actuator at given angle
Torques

Matrix([
[27.1656186753301],
[81.4968560259902]])